# NetID Tutorial

In this tutorial, I would use hematopoietic single cell RNA-seq dataset to show how to conduct network inference through NetID. First, load single cell datasets and the NetID packages

In [10]:
library(NetID)
library(RaceID)
sce <- readRDS("blood_sce.rds")

The sce objects contains the spliced/unspliced read count matrices and the metadata of cells. For showing the performance, I also loaded the transcriptional factors gene sets and the tissue-unspecific GRN curated from ChIP-seq datasets as the ground truth.

In [2]:
sce

class: SingleCellExperiment 
dim: 3439 4742 
metadata(0):
assays(2): spliced unspliced
rownames(3439): X0610007P14Rik X0610010K14Rik ... Zyx l7Rn6
rowData names(0):
colnames(4742): bBM1 bBM2 ... bBM5312 bBM5315
colData names(1): cluster
reducedDimNames(0):
mainExpName: NULL
altExpNames(0):

In [4]:
TF <- read.csv("mouse-tfs.csv",header=TRUE,stringsAsFactors=FALSE)

## 1. Learning GRN skeleton from sketched and aggregated single cell RNA-seq datasets
At the first step, NetID would samples the single cell RNA-seq dataset through sketching methods, e.g. geosketch or Seurat sketch. These sketching methods would sample the cells as the “meta-cells” which would cover all latent manifold of single cell transcriptome. Then, the nearest neighborhoods of those meta-cells would specifically assign to the one meta cell according to the edges P-values calculated from VarID, and finally perform aggregation. The resulted GEP of meta-cells would be used as the input of network inference methods like GENIE3 (default), to learn the basic network structure.

In [5]:
dyn.out <- RunNetID(sce, regulators = TF$TF, targets = TF$TF,netID_params = list(normalize=FALSE), dynamicInfer = FALSE)

Build VarID object...
Using NetID to perform skeleton estimation...
prune sampled neighbourhoods according to q-value...
assign weight for edges using p-value...
aggregated matrix: the number of genes:356; the number of samples:425


Tree method: RF
K: sqrt
Number of trees: 500


Using 12 cores.



Done...


This step only output the global GRN learned from all sampled meta-cells. To learn lineage-specific GRN, we need to run cellrank and Palantir at first. NetID provided a function for end-to-end cell fate analysis through cellrank and Palantir.

In [13]:
FateDynamic(sce,global_params = list(cluster_label = "cluster",min_counts = 10 ,nhvgs = 3000,velo = FALSE),palantir_params = list(start_cell = "bBM1"))

Using palantir to perform cell fate analysis...


NetID provides three methods for identifying root cell, includes stemness score based method: SCENT (scent) and CytoTRACE (cytotrace), or use marekr gene (markergene), in tutorial just randomly set the cell "bBM1" as the root cell, user could manually set the root cell they want.

In [8]:
library(reticulate)
dyn.out <- RunNetID(sce,regulators = TF$TF, targets = TF$TF,netID_params =
                    list(normalize=FALSE), dynamicInfer = TRUE,velo=FALSE)

Find VarID object at local dictionary, Read VarID object...
Using NetID to perform skeleton estimation...
prune sampled neighbourhoods according to q-value...
assign weight for edges using p-value...
aggregated matrix: the number of genes:356; the number of samples:411


Tree method: RF
K: sqrt
Number of trees: 500


Using 12 cores.



Classify lineage for palantir fate prob...
     cluster1     cluster2     cluster3 cluster4  cluster5
3  0.02469504 2.922082e-04 9.106718e-08 0.155587 2.0602990
5 40.49395588 3.422218e+03 1.098090e+07 6.427273 0.4853664
Allow shared cell state between different lineage...
Done...


In NetID we classify the cells according to the fate probability matrix using Gaussian Mixture Model, then we calculate the lineage fate probability fold change to assigh the cluster to the specific lineage

## 2. Learning cell fate specific GRN through granger causal regression model.
The dyn.out object contains the skeleton of global network (learned from all cells without lineage information) and cell fate probability information. NetID would run L2-penalized granger regression model for each target genes to re-calculate the regulatory coefficients of the skeleton, with using cell fate probability as the “time-series”, and then aggregate the learned coefficients with the global coefficients through rank method.

In [9]:
GRN <- FateCausal(dyn.out,velo_infor = FALSE,L=30,aggre_method = "RobustRankAggreg")

Find grn_list object at local dictionary, Read grn_list...
Integrating Network Skeleton And Granger Causal Coefficience...


The output of FateCausal is a list object contains the lineage specific GRN.